# 初始化模型参数

我们仍然用MLP这个例子来详细解释如何初始化模型参数。

In [1]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4, activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))

我们知道如果不`initialize()`直接跑forward，那么系统会抱怨说参数没有初始化。

In [2]:
import sys
try:
    net = get_net()
    net(x)
except RuntimeError as err:
    sys.stderr.write(str(err))

Parameter sequential0_dense0_weight has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

正确的打开方式是这样

In [3]:
net.initialize()
net(x)


[[  5.69327455e-03   5.31650949e-05]
 [  3.97902681e-03   7.88165082e-04]
 [  3.20330053e-03   1.50499865e-03]]
<NDArray 3x2 @cpu(0)>

## 访问模型参数

之前我们提到过可以通过`weight`和`bias`访问`Dense`的参数，他们是`Parameter`这个类：

In [4]:
w = net[0].weight
b = net[0].bias
print('name: ', net[0].name, '\nweight: ', w, '\nbias: ', b)

name:  sequential0_dense0 
weight:  Parameter sequential0_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>) 
bias:  Parameter sequential0_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)


然后我们可以通过`data`来访问参数，`grad`来访问对应的梯度

In [5]:
print('weight:', w.data())
print('weight gradient', w.grad())
print('bias:', b.data())
print('bias gradient', b.grad())

weight: 
[[ 0.01847461 -0.03004881 -0.02461551 -0.01465906 -0.05932271]
 [-0.0595007   0.0434817   0.04195441  0.05774786  0.00482907]
 [ 0.04922146  0.0243923  -0.06268584  0.04367422  0.03679534]
 [-0.06364554  0.03010933  0.05611894 -0.02152951  0.03825361]]
<NDArray 4x5 @cpu(0)>
weight gradient 
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
<NDArray 4x5 @cpu(0)>
bias: 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>
bias gradient 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>


我们也可以通过`collect_params`来访问Block里面所有的参数（这个会包括所有的子Block）。它会返回一个名字到对应Parameter的dict。既可以用正常`[]`来访问参数，也可以用`get()`，它不需要填写名字的前缀。

In [6]:
params = net.collect_params()
print(params)
print(params['sequential0_dense0_bias'].data())
print(params.get('dense0_weight').data())

sequential0_ (
  Parameter sequential0_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_weight (shape=(2, 4), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>

[[ 0.01847461 -0.03004881 -0.02461551 -0.01465906 -0.05932271]
 [-0.0595007   0.0434817   0.04195441  0.05774786  0.00482907]
 [ 0.04922146  0.0243923  -0.06268584  0.04367422  0.03679534]
 [-0.06364554  0.03010933  0.05611894 -0.02152951  0.03825361]]
<NDArray 4x5 @cpu(0)>


## 使用不同的初始函数来初始化

我们一直在使用默认的`initialize`来初始化权重（除了指定GPU `ctx`外）。它会把所有权重初始化成在`[-0.07, 0.07]`之间均匀分布的随机数。我们可以使用别的初始化方法。例如使用均值为0，方差为0.02的正态分布

In [7]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())


[[ 0.01925707  0.00956226  0.0084907   0.03457717 -0.00640247]
 [ 0.03445733  0.04135119 -0.01956459  0.0094062  -0.03433602]
 [ 0.00600536  0.00486903  0.00139216  0.03815848  0.02499754]
 [-0.00589869 -0.03443903 -0.0393823  -0.0066522  -0.01360089]]
<NDArray 4x5 @cpu(0)> 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>


看得更加清楚点：

In [8]:
params.initialize(init=init.One(), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())


[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]
<NDArray 4x5 @cpu(0)> 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>


更多的方法参见[init的API](https://mxnet.incubator.apache.org/api/python/optimization.html#the-mxnet-initializer-package). 下面我们自定义一个初始化方法。

In [9]:
class MyInit(init.Initializer):
    def __init__(self):
        super(MyInit, self).__init__()
        self._verbose = True
    def _init_weight(self, _, arr):
        # 初始化权重，使用out=arr后我们不需指定形状
        print('init weight', arr.shape)
        nd.random.uniform(low=5, high=10, out=arr)
    def _init_bias(self, _, arr):
        print('init bias', arr.shape)
        # 初始化偏移
        arr[:] = 2

# FIXME: init_bias doesn't work
params.initialize(init=MyInit(), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())

init weight (4, 5)
init weight (2, 4)

[[ 6.92873859  5.43979263  9.19637489  6.07963848  7.22268105]
 [ 5.52318239  9.39675808  8.25591183  7.59210968  5.14339209]
 [ 7.52368546  8.17871094  6.14734745  5.996562    6.95708847]
 [ 8.30252838  6.86959934  9.07629967  9.43707752  7.62580872]]
<NDArray 4x5 @cpu(0)> 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>


## 延后的初始化

我们之前提到过Gluon的一个便利的地方是模型定义的时候不需要指定输入的大小，在之后做forward的时候会自动推测参数的大小。我们具体来看这是怎么工作的。

新创建一个网络，然后打印参数。你会发现两个全连接层的权重的形状里都有0。 这是因为在不知道输入数据的情况下，我们无法判断它们的形状。

In [10]:
net = get_net()
print(net.collect_params())

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_weight (shape=(2, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)


然后我们初始化

In [11]:
net.initialize(init=MyInit())

你会看到我们并没有看到MyInit打印的东西，这是因为我们仍然不知道形状。真正的初始化发生在我们看到数据时。

In [12]:
net(x)

init weight (4, 5)
init weight (2, 4)



[[ 702.39978027  624.83215332]
 [ 666.17633057  593.84613037]
 [ 526.71362305  468.61410522]]
<NDArray 3x2 @cpu(0)>

这时候我们看到shape里面的0被填上正确的值了。

In [13]:
print(net.collect_params())

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_weight (shape=(2, 4), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)


## 避免延后初始化

有时候我们不想要延后初始化，这时候可以在创建网络的时候指定输入大小。

In [14]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4, in_units=5, activation="relu"))
    net.add(nn.Dense(2, in_units=4))

net.initialize(MyInit())

init weight (4, 5)
init weight (2, 4)


## 共享模型参数

有时候我们想在层之间共享同一份参数，我们可以通过Block的`params`输出参数来手动指定参数，而不是让系统自动生成。

In [15]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4, in_units=4, activation="relu"))
    net.add(nn.Dense(4, in_units=4, activation="relu", params=net[-1].params))
    net.add(nn.Dense(2, in_units=4))



初始化然后打印

In [16]:
net.initialize(MyInit())
print(net[0].weight.data())
print(net[1].weight.data())

init weight (4, 4)
init weight (2, 4)

[[ 7.72266722  5.12258196  7.70183563  9.58588409]
 [ 5.34961605  5.22156763  6.64358377  5.61120892]
 [ 6.21387911  8.66970825  5.88276386  5.25478649]
 [ 6.20189953  9.93853378  8.26059151  8.12326527]]
<NDArray 4x4 @cpu(0)>

[[ 7.72266722  5.12258196  7.70183563  9.58588409]
 [ 5.34961605  5.22156763  6.64358377  5.61120892]
 [ 6.21387911  8.66970825  5.88276386  5.25478649]
 [ 6.20189953  9.93853378  8.26059151  8.12326527]]
<NDArray 4x4 @cpu(0)>


## 总结

我们可以很灵活地访问和修改模型参数。

## 练习

1. 研究下`net.collect_params()`返回的是什么？`net.params`呢？
1. 如何对每个层使用不同的初始化函数
1. 如果两个层共用一个参数，那么求梯度的时候会发生什么？

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/987)